In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import requests
import datetime
import os
import sys
from copy import copy
sys.path.append('../..')
from data.dataloader import Covid19IndiaLoader
from data.processing import get_data, get_district_time_series, combine_districts, concat_sources, get_concat_data

In [ ]:
loader = Covid19IndiaLoader()
dataframes = loader.get_covid19india_api_data()

In [ ]:
original_copy = copy(dataframes)

In [ ]:
bglr1 = get_data(dataframes, 'Karnataka', 'Bengaluru Rural')

In [ ]:
bglr2 = get_data(dataframes, 'Karnataka', 'Bengaluru Urban')

In [ ]:
bglr_all = combine_districts([bglr1, bglr2], 'Bengaluru')
bglr_all[:3]

In [ ]:
amd1 = get_district_time_series(dataframes, 'Gujarat', 'Ahmedabad')
amd1[:3]

In [ ]:
amd2 = get_district_time_series(dataframes, 'Gujarat', 'Ahmedabad', use_dataframe='deaths_recovs')
amd2[:3]

In [ ]:
amd3 = get_data(dataframes, 'Gujarat', 'Ahmedabad')
amd3[:3]

In [ ]:
amd = concat_sources(amd1, amd2, amd3)

In [ ]:
bglr3 = get_district_time_series(dataframes, 'Karnataka', 'Bengaluru Rural')
bglr4 = get_district_time_series(dataframes, 'Karnataka', 'Bengaluru Urban')
bglr_raw = combine_districts([bglr3, bglr4], 'Bengaluru', 'Karnataka')

In [ ]:
bglr5 = get_district_time_series(dataframes, 'Karnataka', 'Bengaluru Rural', use_dataframe='deaths_recovs')
bglr6 = get_district_time_series(dataframes, 'Karnataka', 'Bengaluru Urban', use_dataframe='deaths_recovs')
bglr_dr = combine_districts([bglr5, bglr6], 'Bengaluru', 'Karnataka')

In [ ]:
bglr = concat_sources(bglr_raw, bglr_dr, bglr_all)

# fixing dtype issue

In [ ]:
amd3.dtypes

In [ ]:
df_districts = copy(dataframes['df_districts'])
df_districts.dtypes

In [ ]:
state = 'Gujarat'
district = 'Ahmedabad'

In [ ]:
df_district = df_districts[np.logical_and(df_districts['state'] == state, df_districts['district'] == district)]
del df_district['notes']
df_district.loc[:, 'date'] = pd.to_datetime(df_district.loc[:, 'date'])
df_district = df_district.loc[df_district['date'] >= '2020-04-24', :]
df_district = df_district.loc[df_district['date'] < datetime.date.today().strftime("%Y-%m-%d"), :]
df_district.columns = [x if x != 'active' else 'hospitalised' for x in df_district.columns]
df_district.columns = [x if x != 'confirmed' else 'total_infected' for x in df_district.columns]
df_district.reset_index(inplace=True, drop=True)
cols = list(set(df_district.columns) - {'state', 'district', 'date'})
df_district[cols] = df_district[cols].apply(pd.to_numeric)
df_district.dtypes

In [ ]:
c = get_concat_data(dataframes, district=['East Delhi', 'South West Delhi', 'West Delhi', 'North Delhi', 'New Delhi', 'South Delhi', 'North East Delhi', 'North West Delhi', ''], state='Delhi', new_district_name='Delhi')

In [ ]:
c

In [ ]:
delhi_dist = get_data(dataframes, 'Delhi', 'New Delhi')

In [ ]:
delhi_dist

In [ ]:
delhi_dist2 = get_data(dataframes, 'Delhi', '')

In [ ]:
delhi_dist2

In [ ]:
for d in ['East Delhi', 'South West Delhi', 'West Delhi', 'North Delhi', 'New Delhi', 'South Delhi', 'North East Delhi', 'North West Delhi', '']:
    print(get_data(dataframes, 'Delhi', d))

In [ ]:
dataframes.keys()

In [ ]:
statewise = dataframes['df_districts']
statewise = statewise[statewise['state'] == 'Delhi']
statewise['district'].unique()

In [ ]:
'East Delhi', 'New Delhi', 'North Delhi', 'North East Delhi','North West Delhi', 'South Delhi', 'South West Delhi','West Delhi', 'Unknown', 'Central Delhi', 'Shahdara','South East Delhi']